# 算法b）三层卷积和池化两层全连接（参数略有变动）

In [3]:
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as Data
import torchvision
import torch.nn.functional as F
import numpy as np
learning_rate = 1e-4
keep_prob_rate = 0.5 #
max_epoch = 20
BATCH_SIZE = 50

DOWNLOAD_MNIST = False

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("using cuda")
else:
    device = torch.device('cpu')
    print("using cuda")

if not(os.path.exists('./mnist/')) or not os.listdir('./mnist/'):
    # not mnist dir or mnist is empyt dir
    DOWNLOAD_MNIST = True
train_data = torchvision.datasets.MNIST(root='./mnist/',train=True, transform=torchvision.transforms.ToTensor(), download=DOWNLOAD_MNIST,)
train_loader = Data.DataLoader(dataset = train_data ,batch_size= BATCH_SIZE ,shuffle= True)

test_data = torchvision.datasets.MNIST(root = './mnist/',train = False)
test_x = Variable(torch.unsqueeze(test_data.data,dim  = 1)).type(torch.cuda.FloatTensor)[:500]/255.
test_y = test_data.targets[:500].numpy()

test_y_list=np.zeros(10)
for item in test_y:
    test_y_list[item]+=1

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                # patch 7 * 7 ; 1  in channels ; 32 out channels ; ; stride is 1
                # padding style is same(that means the convolution opration's input and output have the same size)
                in_channels=1,
                out_channels=64,
                kernel_size=7,
                stride=1,
                padding=3,
            ),
            nn.ReLU(),  # activation function
            nn.MaxPool2d(2),  # pooling operation
        )
        self.conv2 = nn.Sequential(  # ???
            # line 1 : convolution function, patch 5*5 , 32 in channels ;64 out channels; padding style is same; stride is 1
            # line 2 : choosing your activation funciont
            # line 3 : pooling operation function.
            nn.Conv2d(  # ???
                in_channels=32,
                out_channels=64,
                kernel_size=5,
                stride=1,
                padding=2,
            ),
            nn.ReLU(),  # activation function
            nn.MaxPool2d(2),  # pooling operation
        )
        ##3 4 replace 2
        self.conv3 = nn.Sequential(  # ???
            # line 1 : convolution function, patch 5*5 , 32 in channels ;64 out channels; padding style is same; stride is 1
            # line 2 : choosing your activation funciont
            # line 3 : pooling operation function.
            nn.Conv2d(
                in_channels=64,
                out_channels=128,
                kernel_size=3,
                stride=1,
                padding=1,
            ),
            nn.ReLU(),  # activation function
            nn.MaxPool2d(2),  # pooling operation
        )
        self.conv4 = nn.Sequential(  # ???
            # line 1 : convolution function, patch 5*5 , 32 in channels ;64 out channels; padding style is same; stride is 1
            # line 2 : choosing your activation funciont
            # line 3 : pooling operation function.
            nn.Conv2d(
                in_channels=128,
                out_channels=256,
                kernel_size=3,
                stride=1,
                padding=1,
            ),
            nn.ReLU(),  # activation function
            nn.MaxPool2d(2),  # pooling operation
        )
        self.out1 =nn.Sequential(
            nn.Linear(2304, 1024, bias=True), # full connect
            nn.ReLU()
        )

        self.dropout = nn.Dropout(keep_prob_rate)
        self.out2 = nn.Linear(1024, 10, bias=True)

    def forward(self, x):
        x = self.conv1(x)

        # x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = x.view(x.size(0), 2304)  # flatten the output of coonv2 to (batch_size ,64 * 7 * 7)
        out1 = self.out1(x) # full connect1
        # out1 = F.relu(out1)

        out1 = self.dropout(out1)
        out2 = self.out2(out1)
        output = F.softmax(out2)
        return output

def test(cnn):
    predict = np.ones(10)# to avoid 0 being denominator(分母)
    TP = np.zeros(10)
    y_pre = cnn(test_x)
    _, pre_index = torch.max(y_pre, 1)#max return first is the max value of the line,the second is the index of the max value in the line.
    pre_index = pre_index.view(-1)
    pre_index_cpu = pre_index.cpu()
    prediction = pre_index_cpu.data.numpy()
    for index, item in enumerate(prediction):
        predict[item]+=1
        if prediction[index]==test_y[index]:
            TP[item]+=1
    precision = TP/predict
    recall = TP/test_y_list
    F1 = 2*precision*recall/(precision+recall)
    correct = np.sum(prediction == test_y)
    correct = correct/500
    return F1, correct


def train(cnn):
    optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)
    loss_func = nn.CrossEntropyLoss()
    for epoch in range(max_epoch):
        for step, (x_, y_) in enumerate(train_loader):
            x, y = Variable(x_).to(device), Variable(y_).to(device)
            output = cnn(x)
            loss = loss_func(output, y)
            optimizer.zero_grad()# clear the parameters such as weights and bias, for the function 'backward' would add the new to the last ones.
            loss.backward()
            optimizer.step()# conduct the gradient descent using the gradient generated in the 'backward'.

            # if step != 0 and step % 100 == 0:
            #     F1,correct = test(cnn)
            #     np.set_printoptions(formatter={'float': '{: 0.3f}'.format})
            #     print("=" * 10, step, "=" * 5, "test F1 is ",F1, "=" * 5,"test sum accuracy is ", correct)
        F1, correct = test(cnn)
        np.set_printoptions(formatter={'float': '{: 0.3f}'.format})
        print("=" * 10, "epoch:",epoch, "=" * 5, "test F1 is ", F1, "=" * 5, "test sum accuracy is ", correct)
if __name__ == '__main__':

    cnn = CNN()
    cnn.to(device)
    train(cnn)


using cuda


D:\SoftwareFamily\Anaconda\envs\nlplab\lib\site-packages\ipykernel_launcher.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


========== epoch: 0 ===== test F1 is  [ 0.966  0.993  0.982  0.956  0.938  0.961  0.953  0.940  0.884  0.889] ===== test sum accuracy is  0.958
========== epoch: 1 ===== test F1 is  [ 0.955  0.993  0.982  0.978  0.982  0.980  0.965  0.990  0.940  0.962] ===== test sum accuracy is  0.984
========== epoch: 2 ===== test F1 is  [ 0.953  0.978  0.956  0.978  0.972  0.980  0.953  0.970  0.951  0.962] ===== test sum accuracy is  0.976
========== epoch: 3 ===== test F1 is  [ 0.966  0.985  0.982  0.978  0.973  0.980  0.977  0.990  0.962  0.981] ===== test sum accuracy is  0.988
========== epoch: 4 ===== test F1 is  [ 0.977  0.985  0.982  0.978  0.972  0.980  0.977  0.990  0.988  0.972] ===== test sum accuracy is  0.99
========== epoch: 5 ===== test F1 is  [ 0.976  0.993  0.964  0.978  0.963  0.970  0.989  0.980  0.975  0.973] ===== test sum accuracy is  0.986
========== epoch: 6 ===== test F1 is  [ 0.966  0.993  0.973  0.978  0.982  0.970  0.952  0.980  0.976  0.981] ===== test sum accuracy is 

# 算法c) 引入ResNet残差思想

In [5]:
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as Data
import torchvision
import torch.nn.functional as F
import numpy as np
learning_rate = 1e-4
keep_prob_rate = 0.2 #
max_epoch = 20
BATCH_SIZE = 50

DOWNLOAD_MNIST = False

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("using cuda")
else:
    device = torch.device('cpu')
    print("using cuda")

if not(os.path.exists('./mnist/')) or not os.listdir('./mnist/'):
    # not mnist dir or mnist is empyt dir
    DOWNLOAD_MNIST = True
train_data = torchvision.datasets.MNIST(root='./mnist/',train=True, transform=torchvision.transforms.ToTensor(), download=DOWNLOAD_MNIST,)
train_loader = Data.DataLoader(dataset = train_data ,batch_size= BATCH_SIZE ,shuffle= True)

test_data = torchvision.datasets.MNIST(root = './mnist/',train = False)
test_x = Variable(torch.unsqueeze(test_data.data,dim  = 1)).type(torch.cuda.FloatTensor)[:500]/255.
test_y = test_data.targets[:500].numpy()

test_y_list=np.zeros(10)
for item in test_y:
    test_y_list[item]+=1

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=1,
                out_channels=64,
                kernel_size=7,
                stride=1,
                padding=3,
            ),
            nn.ReLU(),  # activation function
            nn.MaxPool2d(2),  # pooling operation
        )
        self.conv2 = nn.Sequential(  # ???
          
            nn.Conv2d(  # ???
                in_channels=64,
                out_channels=64,
                kernel_size=3,
                stride=1,
                padding=1,
            )
        )
        ##3 4 replace 2
        self.conv3 = nn.Sequential(  # ???
            # line 1 : convolution function, patch 5*5 , 32 in channels ;64 out channels; padding style is same; stride is 1
            # line 2 : choosing your activation funciont
            # line 3 : pooling operation function.
            nn.Conv2d(
                in_channels=64,
                out_channels=256,
                kernel_size=3,
                stride=1,
                padding=1,
            ),
            nn.ReLU(),
            nn.MaxPool2d(2), # pooling operation
        )
        self.conv4 = nn.Sequential(  # ???
            # line 1 : convolution function, patch 5*5 , 32 in channels ;64 out channels; padding style is same; stride is 1
            # line 2 : choosing your activation funciont
            # line 3 : pooling operation function.
            nn.Conv2d(
                in_channels=256,
                out_channels=256,
                kernel_size=3,
                stride=1,
                padding=1,
            ),
        )
        self.conv5 = nn.Sequential(
            # line 1 : convolution function, patch 5*5 , 32 in channels ;64 out channels; padding style is same; stride is 1
            # line 2 : choosing your activation funciont
            # line 3 : pooling operation function.
            nn.Conv2d(
                in_channels=64,
                out_channels=256,
                kernel_size=1,
                stride=2,
                padding=0,
            ),
            # nn.ReLU(),  # activation function
            # nn.MaxPool2d(2),  # pooling operation
        )

        self.out1 =nn.Sequential(
            nn.Linear(12544, 1024, bias=True), # full connect
            nn.ReLU()
        )

        self.dropout = nn.Dropout(keep_prob_rate)
        self.out2 = nn.Linear(1024, 10, bias=True)

    def forward(self, x):
        x1 = self.conv1(x)#包含 relu 和 最大池化

        x2_1 = self.conv2(x1)
        x2_1 = F.relu(x2_1)
        x2_1 = self.conv2(x2_1)
        x2_1 = F.relu(x1 + x2_1)

        x2_2 = self.conv2(x2_1)
        x2_2 = F.relu(x2_2)
        x2_2 = self.conv2(x2_2)
        x2_2 = F.relu(x2_1 + x2_2)

        x3_1 = self.conv3(x2_2)# 包含 relu 和 最大池化
        x3_1 = self.conv4(x3_1)
        x2_2 = self.conv5(x2_2)# 通道数转换64-256
        x3_1 = F.relu(x3_1 + x2_2)

        x3_2 = self.conv4(x3_1)
        x3_2 = F.relu(x3_2)
        x3_2 = self.conv4(x3_2)
        x3_2 = F.relu(x3_1 + x3_2)

        x3_2 = x3_2.view(x3_2.size(0), 12544)  # flatten the output of coonv2 to (batch_size ,64 * 7 * 7)
        out1 = self.out1(x3_2) # full connect1

        out1 = self.dropout(out1)
        out2 = self.out2(out1)
        output = F.softmax(out2)
        return output


def test(cnn):
    predict = np.ones(10)# to avoid 0 being denominator(分母)
    TP = np.zeros(10)
    y_pre = cnn(test_x)
    _, pre_index = torch.max(y_pre, 1)#max return first is the max value of the line,the second is the index of the max value in the line.
    pre_index = pre_index.view(-1)
    pre_index_cpu = pre_index.cpu()
    prediction = pre_index_cpu.data.numpy()
    for index, item in enumerate(prediction):
        predict[item]+=1
        if prediction[index]==test_y[index]:
            TP[item]+=1
    precision = TP/predict
    recall = TP/test_y_list
    F1 = 2*precision*recall/(precision+recall)
    correct = np.sum(prediction == test_y)
    correct = correct/500
    return F1, correct


def train(cnn):
    optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)
    loss_func = nn.CrossEntropyLoss()
    for epoch in range(max_epoch):
        for step, (x_, y_) in enumerate(train_loader):
            x, y = Variable(x_).to(device), Variable(y_).to(device)
            output = cnn(x)
            loss = loss_func(output, y)
            optimizer.zero_grad()# clear the parameters such as weights and bias, for the function 'backward' would add the new to the last ones.
            loss.backward()
            optimizer.step()# conduct the gradient descent using the gradient generated in the 'backward'.

            # if step != 0 and step % 100 == 0:
            #     F1,correct = test(cnn)
            #     np.set_printoptions(formatter={'float': '{: 0.3f}'.format})
            #     print("=" * 10, step, "=" * 5, "test F1 is ",F1, "=" * 5,"test sum accuracy is ", correct)
        F1, correct = test(cnn)
        np.set_printoptions(formatter={'float': '{: 0.3f}'.format})
        print("=" * 10, "epoch:",epoch, "=" * 5, "test F1 is ", F1, "=" * 5, "test sum accuracy is ", correct)
if __name__ == '__main__':

    cnn = CNN()
    cnn.to(device)
    train(cnn)


using cuda


D:\SoftwareFamily\Anaconda\envs\nlplab\lib\site-packages\ipykernel_launcher.py:140: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
D:\SoftwareFamily\Anaconda\envs\nlplab\lib\site-packages\ipykernel_launcher.py:158: RuntimeWarning: invalid value encountered in true_divide


========== epoch: 0 ===== test F1 is  [ 0.977  0.985  0.954  0.944  0.828  0.942  0.977  0.760  0.837  nan] ===== test sum accuracy is  0.866
========== epoch: 1 ===== test F1 is  [ 0.977  0.993  0.964  0.955  0.944  0.971  0.944  0.914  0.963  0.933] ===== test sum accuracy is  0.966
========== epoch: 2 ===== test F1 is  [ 0.955  0.957  0.933  0.978  0.972  0.980  0.932  0.980  0.962  0.991] ===== test sum accuracy is  0.974
========== epoch: 3 ===== test F1 is  [ 0.955  0.993  0.982  0.967  0.991  0.970  0.965  0.980  0.962  0.982] ===== test sum accuracy is  0.986
========== epoch: 4 ===== test F1 is  [ 0.976  0.993  0.982  0.989  0.982  0.971  0.966  0.980  0.975  0.982] ===== test sum accuracy is  0.99
========== epoch: 5 ===== test F1 is  [ 0.966  0.993  0.973  0.957  0.956  0.959  0.953  0.990  0.950  0.953] ===== test sum accuracy is  0.976
========== epoch: 6 ===== test F1 is  [ 0.988  0.993  0.991  0.978  0.972  0.980  0.977  0.990  0.976  0.972] ===== test sum accuracy is  0

# 算法d)更换激活函数

In [6]:
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as Data
import torchvision
import torch.nn.functional as F
import numpy as np
learning_rate = 1e-4
keep_prob_rate = 0.2 #
max_epoch = 20
BATCH_SIZE = 50

DOWNLOAD_MNIST = False

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("using cuda")
else:
    device = torch.device('cpu')
    print("using cuda")

if not(os.path.exists('./mnist/')) or not os.listdir('./mnist/'):
    # not mnist dir or mnist is empyt dir
    DOWNLOAD_MNIST = True
train_data = torchvision.datasets.MNIST(root='./mnist/',train=True, transform=torchvision.transforms.ToTensor(), download=DOWNLOAD_MNIST,)
train_loader = Data.DataLoader(dataset = train_data ,batch_size= BATCH_SIZE ,shuffle= True)

test_data = torchvision.datasets.MNIST(root = './mnist/',train = False)
test_x = Variable(torch.unsqueeze(test_data.data,dim  = 1)).type(torch.cuda.FloatTensor)[:500]/255.
test_y = test_data.targets[:500].numpy()

test_y_list=np.zeros(10)
for item in test_y:
    test_y_list[item]+=1

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                # patch 7 * 7 ; 1  in channels ; 32 out channels  ; stride is 1
                # padding style is same(that means the convolution opration's input and output have the same size)
                in_channels=1,
                out_channels=64,
                kernel_size=7,
                stride=1,
                padding=3,
            ),
            nn.Sigmoid(),
            # nn.ReLU(),  # activation function
            nn.MaxPool2d(2),  # pooling operation
        )
        self.conv2 = nn.Sequential(  # ???
            # line 1 : convolution function, patch 5*5 , 32 in channels ;64 out channels; padding style is same; stride is 1
            # line 2 : choosing your activation funciont
            # line 3 : pooling operation function.
            nn.Conv2d(  # ???
                in_channels=64,
                out_channels=64,
                kernel_size=3,
                stride=1,
                padding=1,
            )
        )
        ##3 4 replace 2
        self.conv3 = nn.Sequential(  # ???
            # line 1 : convolution function, patch 5*5 , 32 in channels ;64 out channels; padding style is same; stride is 1
            # line 2 : choosing your activation funciont
            # line 3 : pooling operation function.
            nn.Conv2d(
                in_channels=64,
                out_channels=256,
                kernel_size=3,
                stride=1,
                padding=1,
            ),
            nn.Sigmoid(),
            # nn.ReLU(),
            nn.MaxPool2d(2), # pooling operation
        )
        self.conv4 = nn.Sequential(  # ???
            # line 1 : convolution function, patch 5*5 , 32 in channels ;64 out channels; padding style is same; stride is 1
            # line 2 : choosing your activation funciont
            # line 3 : pooling operation function.
            nn.Conv2d(
                in_channels=256,
                out_channels=256,
                kernel_size=3,
                stride=1,
                padding=1,
            ),
        )
        self.conv5 = nn.Sequential(
            # line 1 : convolution function, patch 5*5 , 32 in channels ;64 out channels; padding style is same; stride is 1
            # line 2 : choosing your activation funciont
            # line 3 : pooling operation function.
            nn.Conv2d(
                in_channels=64,
                out_channels=256,
                kernel_size=1,
                stride=2,
                padding=0,
            ),
        )

        self.out1 =nn.Sequential(
            nn.Linear(12544, 1024, bias=True), # full connect
            # nn.ReLU()
            nn.Sigmoid()
        )

        self.dropout = nn.Dropout(keep_prob_rate)
        self.out2 = nn.Linear(1024, 10, bias=True)

    def forward(self, x):
        x1 = self.conv1(x)#包含 relu 和 最大池化

        x2_1 = self.conv2(x1)
        x2_1 = F.relu(x2_1)

        x2_1 = self.conv2(x2_1)
        x2_1 = F.relu(x1 + x2_1)

        x2_2 = self.conv2(x2_1)
        x2_2 = F.relu(x2_2)
        x2_2 = self.conv2(x2_2)
        x2_2 = F.relu(x2_1 + x2_2)

        x3_1 = self.conv3(x2_2)# 包含 relu 和 最大池化
        x3_1 = self.conv4(x3_1)
        x2_2 = self.conv5(x2_2)# 通道数转换64-256
        x3_1 = F.relu(x3_1 + x2_2)

        x3_2 = self.conv4(x3_1)
        x3_2 = F.relu(x3_2)
        x3_2 = self.conv4(x3_2)
        x3_2 = F.relu(x3_1 + x3_2)

        x3_2 = x3_2.view(x3_2.size(0), 12544)  # flatten the output of coonv2 to (batch_size ,64 * 7 * 7)
        out1 = self.out1(x3_2) # full connect1

        out1 = self.dropout(out1)
        out2 = self.out2(out1)
        output = F.softmax(out2)
        return output


def test(cnn):
    predict = np.ones(10)# to avoid 0 being denominator(分母)
    TP = np.zeros(10)
    y_pre = cnn(test_x)
    _, pre_index = torch.max(y_pre, 1)#max return first is the max value of the line,the second is the index of the max value in the line.
    pre_index = pre_index.view(-1)
    pre_index_cpu = pre_index.cpu()
    prediction = pre_index_cpu.data.numpy()
    for index, item in enumerate(prediction):
        predict[item]+=1
        if prediction[index]==test_y[index]:
            TP[item]+=1
    precision = TP/predict
    recall = TP/test_y_list
    F1 = 2*precision*recall/(precision+recall)
    correct = np.sum(prediction == test_y)
    correct = correct/500
    return F1, correct


def train(cnn):
    optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)
    loss_func = nn.CrossEntropyLoss()
    for epoch in range(max_epoch):
        for step, (x_, y_) in enumerate(train_loader):
            x, y = Variable(x_).to(device), Variable(y_).to(device)
            output = cnn(x)
            loss = loss_func(output, y)
            optimizer.zero_grad()# clear the parameters such as weights and bias, for the function 'backward' would add the new to the last ones.
            loss.backward()
            optimizer.step()# conduct the gradient descent using the gradient generated in the 'backward'.

            # if step != 0 and step % 100 == 0:
            #     F1,correct = test(cnn)
            #     np.set_printoptions(formatter={'float': '{: 0.3f}'.format})
            #     print("=" * 10, step, "=" * 5, "test F1 is ",F1, "=" * 5,"test sum accuracy is ", correct)
        F1, correct = test(cnn)
        np.set_printoptions(formatter={'float': '{: 0.3f}'.format})
        print("=" * 10, "epoch:",epoch, "=" * 5, "test F1 is ", F1, "=" * 5, "test sum accuracy is ", correct)
if __name__ == '__main__':

    cnn = CNN()
    cnn.to(device)
    train(cnn)


using cuda


D:\SoftwareFamily\Anaconda\envs\nlplab\lib\site-packages\ipykernel_launcher.py:146: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


========== epoch: 0 ===== test F1 is  [ 0.955  0.993  0.973  0.936  0.973  0.960  0.965  0.970  0.914  0.942] ===== test sum accuracy is  0.97
========== epoch: 1 ===== test F1 is  [ 0.966  0.993  0.982  0.967  0.964  0.980  0.977  0.980  0.938  0.963] ===== test sum accuracy is  0.982
========== epoch: 2 ===== test F1 is  [ 0.977  0.985  0.973  0.967  0.954  0.980  0.989  0.990  0.923  0.972] ===== test sum accuracy is  0.982
========== epoch: 3 ===== test F1 is  [ 0.977  0.993  0.991  0.978  0.973  0.980  0.977  0.990  0.950  0.981] ===== test sum accuracy is  0.99
========== epoch: 4 ===== test F1 is  [ 0.966  0.985  0.982  0.957  0.953  0.970  0.965  0.990  0.976  0.955] ===== test sum accuracy is  0.98
========== epoch: 5 ===== test F1 is  [ 0.977  0.993  0.991  0.967  0.982  0.980  0.977  0.990  0.951  0.981] ===== test sum accuracy is  0.99
========== epoch: 6 ===== test F1 is  [ 0.988  0.993  0.991  0.967  0.982  0.980  0.977  0.990  0.976  0.991] ===== test sum accuracy is  0.